In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
data = pd.read_csv('/kaggle/input/car-price-prediction/CarPrice_Assignment.csv')
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [3]:
data.drop('car_ID',axis=1,inplace=True)
data.drop('symboling',axis=1,inplace=True)
data.drop('CarName',axis=1,inplace=True)
data.drop('fueltype',axis=1,inplace=True)
data.drop('aspiration',axis=1,inplace=True)
data.drop('doornumber',axis=1,inplace=True)
data.drop('carbody',axis=1,inplace=True)
data.drop('drivewheel',axis=1,inplace=True)
data.drop('enginelocation',axis=1,inplace=True)
data.drop('fuelsystem',axis=1,inplace=True)
data.drop('enginetype',axis=1,inplace=True)
data['cylindernumber'].replace({'three':3,'four':4,'five':5,'six':6,'twelve':12,'eight':6,'two':2},inplace=True)
data['cylindernumber']=data['cylindernumber'].astype(str).astype(int)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   wheelbase         205 non-null    float64
 1   carlength         205 non-null    float64
 2   carwidth          205 non-null    float64
 3   carheight         205 non-null    float64
 4   curbweight        205 non-null    int64  
 5   cylindernumber    205 non-null    int64  
 6   enginesize        205 non-null    int64  
 7   boreratio         205 non-null    float64
 8   stroke            205 non-null    float64
 9   compressionratio  205 non-null    float64
 10  horsepower        205 non-null    int64  
 11  peakrpm           205 non-null    int64  
 12  citympg           205 non-null    int64  
 13  highwaympg        205 non-null    int64  
 14  price             205 non-null    float64
dtypes: float64(8), int64(7)
memory usage: 24.1 KB


In [5]:
#Centralize Values
def toCentralizeInput(_data):
  for col in _data.columns:
    if(col != 'price'):
      _data[col] = [x - data[col].mean()/_data[col].std() for x in _data[col]]

toCentralizeInput(data)
data.head()

,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,72.200089,154.692421,33.37667,26.813344,2543.091871,-0.543263,126.952471,-8.824013,-7.700885,6.446517,108.367069,4989.255186,17.145068,22.534528,13495.0
1,72.200089,154.692421,33.37667,26.813344,2543.091871,-0.543263,126.952471,-8.824013,-7.700885,6.446517,108.367069,4989.255186,17.145068,22.534528,16500.0
2,78.100089,157.092421,34.77667,30.413344,2818.091871,1.456737,148.952471,-9.614013,-6.910885,6.446517,151.367069,4989.255186,15.145068,21.534528,16500.0
3,83.400089,162.492421,35.47667,32.313344,2332.091871,-0.543263,105.952471,-9.104013,-6.980885,7.446517,99.367069,5489.255186,20.145068,25.534528,13950.0
4,83.000089,162.492421,35.67667,32.313344,2819.091871,0.456737,132.952471,-9.104013,-6.980885,5.446517,112.367069,5489.255186,14.145068,17.534528,17450.0


In [6]:
train_X = data.sample(frac=0.8,random_state=0)
val_X = data.drop(train_X.index)
test_X = train_X.sample(frac=0.3,random_state=0)

train_Y = train_X['price']
train_X.drop('price',axis=1,inplace=True)

val_Y = val_X['price']
val_X.drop('price',axis=1,inplace=True)

test_Y = test_X['price']
test_X.drop('price',axis=1,inplace=True)

In [7]:
train_X = train_X.values
train_Y = train_Y.values

test_X = test_X.values
test_Y = test_Y.values

val_X = val_X.values
val_Y = val_Y.values

In [8]:
train_X

array([[  76.70008899,  144.99242134,   33.47667047, ..., 4989.25518613,
          27.14506785,   33.53452792],
       [  88.10008899,  173.69242134,   35.77667047, ..., 5189.25518613,
          15.14506785,   19.53452792],
       [  83.40008899,  163.19242134,   35.57667047, ..., 5489.25518613,
          15.14506785,   20.53452792],
       ...,
       [  86.90008899,  160.49242134,   33.87667047, ..., 4989.25518613,
          20.14506785,   25.53452792],
       [  97.80008899,  184.79242134,   37.67667047, ..., 4139.25518613,
          21.14506785,   20.53452792],
       [  79.30008899,  144.59242134,   32.87667047, ..., 4789.25518613,
          27.14506785,   33.53452792]])

In [9]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(10, activation='relu',
                       input_shape=(train_X.shape[1],)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(6, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [10]:
model.fit(train_X, train_Y,
              epochs=100, batch_size=10)

Epoch 1/100
17/17 [==============================] - 1s 2ms/step - loss: 208999056.0000 - mae: 12574.0739
Epoch 2/100
17/17 [==============================] - 0s 2ms/step - loss: 189611032.0000 - mae: 11652.8187
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: 195358144.0000 - mae: 11743.9904
Epoch 4/100
17/17 [==============================] - 0s 2ms/step - loss: 203876487.1111 - mae: 11905.6032
Epoch 5/100
17/17 [==============================] - 0s 2ms/step - loss: 232219235.5556 - mae: 12605.8510
Epoch 6/100
17/17 [==============================] - 0s 2ms/step - loss: 163005033.3333 - mae: 10847.9046
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: 204251152.0000 - mae: 11820.3980
Epoch 8/100
17/17 [==============================] - 0s 2ms/step - loss: 179965745.3333 - mae: 11151.2983
Epoch 9/100
17/17 [==============================] - 0s 3ms/step - loss: 163542996.0000 - mae: 10441.4042
Epoch 10/100
17/17 [==========================

In [11]:
val_mse, val_mae = model.evaluate(val_X, val_Y)

2/2 [==============================] - 0s 4ms/step - loss: 30286108.0000 - mae: 4165.3257


In [12]:
test_mse,test_mae = model.evaluate(test_X,test_Y)

2/2 [==============================] - 0s 5ms/step - loss: 11232853.0000 - mae: 2949.5383


In [13]:
test_mae

2949.538330078125

In [14]:
model.predict(test_X)

array([[14390.87 ],
       [11278.966],
       [10967.566],
       [11985.775],
       [15303.976],
       [12793.836],
       [11105.807],
       [11493.234],
       [10965.262],
       [13383.703],
       [12081.057],
       [12762.986],
       [15225.332],
       [10474.418],
       [12811.251],
       [14968.254],
       [15158.393],
       [16990.71 ],
       [10403.982],
       [18396.205],
       [10296.845],
       [10131.018],
       [12412.171],
       [17094.018],
       [13662.986],
       [16747.773],
       [17271.672],
       [15834.59 ],
       [12578.133],
       [10015.744],
       [ 8914.273],
       [14686.404],
       [12150.348],
       [16444.434],
       [17865.541],
       [19994.746],
       [10573.223],
       [12376.459],
       [10445.434],
       [13734.27 ],
       [12554.949],
       [13662.986],
       [ 9841.892],
       [12800.631],
       [13136.195],
       [13475.652],
       [11368.046],
       [11841.213],
       [16239.622]], dtype=float32)

In [15]:
test_Y[0]

18280.0

In [16]:
##End